In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from datetime import timedelta
from time import time
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import statsmodels.api as sm
from prophet import Prophet
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

In [2]:
csv_file = r'C:\\Users\\bella\\Downloads\\Y2Q4 JBG050 Data Challenge 2\\total_barnet.csv'
dataset = pd.read_csv(csv_file)
dataset = dataset.drop(['last_outcome_category', 'location'], axis=1)
mask = dataset['ward_name']!=dataset['ward_name']
dataset = dataset[~mask]

In [3]:
dataset.head()
# print(len(dataset))

,crime_id,month,longitude,latitude,lsoa_code,lsoa_name,ward_code,ward_name
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,2010-12,-0.201877,51.655538,E01000248,Barnet 001A,E05013644,High Barnet
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,2010-12,-0.207853,51.654317,E01000248,Barnet 001A,E05013644,High Barnet
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2010-12,-0.202510,51.656348,E01000248,Barnet 001A,E05013644,High Barnet
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,2010-12,-0.206779,51.654768,E01000248,Barnet 001A,E05013644,High Barnet
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,2010-12,-0.209537,51.655223,E01000249,Barnet 001B,E05013644,High Barnet


In [8]:
# sarima function

def prepare_timeseries(timeseries):
    
    data_start = date(2010, 12, 1)
    data_end = date(2022, 9, 1)
    timeseries = pd.Series(timeseries.values,index=[pd.Timestamp(x, freq='M') for x in timeseries.index])
    timeseries = timeseries[data_start:data_end]
    
    return timeseries

def training(timeseries):

    train_end = date(2022, 1, 1)
    test_end = date(2022, 9, 1)

    train_data = timeseries[:train_end]
    test_data = timeseries[train_end+timedelta(days=1):test_end]
    
    return train_data, test_data

def corona_troubles(timeseries):
    remove_start = date(2019, 11, 1)
    remove_end = date(2020, 7, 1)
    temp_df = pd.DataFrame(timeseries).reset_index()
    
    
    for x in timeseries[remove_start:remove_end].index:
        temp = temp_df[temp_df['index'].dt.month == x.month]
        temp = temp[temp['index'].dt.year != x.year]
        timeseries[x] = int(temp[0].mean())
        
    return timeseries

def create_model(train_data, my_order, my_seasonal_order):
    my_order = (0,1,0)
    my_seasonal_order = (2, 0, 1, 6)
    model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order)
    
    start = time()
    model_fit = model.fit()
    end = time()
#     print("Model Fitting Time:", end - start)
    
    return model_fit

def pred_res(model_fit, test_data):
    
    predictions = model_fit.forecast(len(test_data))
    predictions = pd.Series(predictions, index=test_data.index)
    residuals = test_data - predictions
    
    return predictions, residuals

def plot_residuals(residuals):
    plt.figure(figsize=(10,4))
    plt.plot(residuals)
    plt.axhline(0, linestyle='--', color='k')
    plt.title('Residuals from SARIMA Model', fontsize=20)
    plt.ylabel("Error", fontsize=16);
    
def plot_predictions(timeseries, predictions):
    plt.figure(figsize=(10,4))

    start_date = timeseries.index[0]
    end_date = timeseries.index[-1]

    plt.plot(timeseries)
    plt.plot(predictions)

    plt.legend(('Data', 'Predictions'), fontsize=16)

    plt.title('Amount of crimes in Barnet', fontsize=20)
    plt.ylabel('Crimes', fontsize=16)
    for year in range(start_date.year+1,end_date.year+1):
        plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
        
def statistics(residuals, test_data):
#     print('Mean Absolute Percent Error:', round(np.mean(abs(residuals/test_data)),4))
#     print('Root Mean Squared Error:', np.sqrt(np.mean(residuals**2)))
    MAPE = round(np.mean(abs(residuals/test_data)),4)
    RMSE = np.sqrt(np.mean(residuals**2))
    return MAPE, RMSE
    
def rolling_pred_res(train_data, test_data, timeseries, my_order, my_seasonal_order):
    my_order = (0,1,0)
    my_seasonal_order = (2, 0, 1, 6)
    rolling_predictions = test_data.copy()
    for train_end in test_data.index:
        train_data = timeseries[:train_end-timedelta(days=1)]
        model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order)
        model_fit = model.fit()

        pred = model_fit.forecast()
        rolling_predictions[train_end] = pred

    rolling_residuals = test_data - rolling_predictions
    
    return rolling_predictions, rolling_residuals, model_fit

def plot_rolling_residuals(rolling_residuals):
    plt.figure(figsize=(10,4))
    plt.plot(rolling_residuals)
    plt.axhline(0, linestyle='--', color='k')
    plt.title('Rolling Forecast Residuals from SARIMA Model', fontsize=20)
    plt.ylabel('Error', fontsize=16);
    

def plot_rolling_predictions(timeseries, rolling_predictions):
    plt.figure(figsize=(10,4))
    
    start_date = timeseries.index[0]
    end_date = timeseries.index[-1]

    plt.plot(timeseries)
    plt.plot(rolling_predictions)

    plt.legend(('Data', 'Predictions'), fontsize=16)

    plt.title('Rolling Forecast prediction Crimes in Barnet', fontsize=20)
    plt.ylabel('Crimes', fontsize=16)
    for year in range(start_date.year+1,end_date.year+1):
        plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
        
        
def rolling_statistics(rolling_residuals, test_data):
#     print('Rolling Mean Absolute Percent Error:', round(np.mean(abs(rolling_residuals/test_data)),4))
#     print('Rolling Root Mean Squared Error:', np.sqrt(np.mean(rolling_residuals**2)))
    RMAPE = round(np.mean(abs(rolling_residuals/test_data)),4)
    RRMSE = np.sqrt(np.mean(rolling_residuals**2))
    
    return RMAPE, RRMSE

# def sarima_model(temp_timeseries):
#     temp_timeseries = prepare_timeseries(temp_timeseries)
#     temp_timeseries = corona_troubles(temp_timeseries)
#     temp_train, temp_test = training(temp_timeseries)
    
#     temp_model_fit = create_model(temp_train, my_order, my_seasonal_order)
#     temp_predictions, temp_residuals = pred_res(temp_model_fit, temp_test)
#     # plot_residuals(temp_residuals)
#     # plot_predictions(temp_timeseries, temp_predictions)
#     temp_mape, temp_rmse = statistics(temp_residuals, temp_test)

#     temp_rolling_predictions, temp_rolling_residuals, model_fit = rolling_pred_res(temp_train, temp_test, temp_timeseries, my_order, my_seasonal_order)
#     # plot_rolling_residuals(temp_rolling_residuals)
#     # plot_rolling_predictions(temp_timeseries, temp_rolling_predictions)
#     temp_rmape, temp_rrmse = rolling_statistics(temp_rolling_residuals, temp_test)
    
#     return temp_rmape, temp_rrmse, model_fit

def sarima_model(ward_name):
    dataframe_ward = dataset[dataset['ward_name']==ward_name]
    temp_timeseries = pd.DataFrame()
    temp_timeseries = dataframe_ward.groupby("month").count()['latitude']
    my_order = (0,0,0)
    my_seasonal_order = (2,0,0,6)
    temp_timeseries = prepare_timeseries(temp_timeseries)
    temp_timeseries = corona_troubles(temp_timeseries)
    temp_train, temp_test = training(temp_timeseries)
    
    temp_model_fit = create_model(temp_train, my_order, my_seasonal_order)
    temp_predictions, temp_residuals = pred_res(temp_model_fit, temp_test)
    # plot_residuals(temp_residuals)
    # plot_predictions(temp_timeseries, temp_predictions)
    temp_mape, temp_rmse = statistics(temp_residuals, temp_test)

    temp_rolling_predictions, temp_rolling_residuals, model_fit = rolling_pred_res(temp_train, temp_test, temp_timeseries, my_order, my_seasonal_order)
    # plot_rolling_residuals(temp_rolling_residuals)
    # plot_rolling_predictions(temp_timeseries, temp_rolling_predictions)
    temp_rmape, temp_rrmse = rolling_statistics(temp_rolling_residuals, temp_test)
    
    return temp_rmape, temp_rrmse, model_fit



# example usage
ward = 'Burnt Oak'
# my_order = (0,0,0)
# my_seasonal_order = (2,0,0,6)
# df_temp = dataset[dataset['ward_name'] == ward]
# temp_timeseries = df_temp.groupby("month").count()['latitude']
# sarima_mape, sarima_rmse, model_fit = sarima_model('Underhill')
sarima_mape, sarima_rmse, model_fit = sarima_model(ward)
sarima_mape, sarima_rmse, model_fit.forecast()[0]

(0.9493, 9.37724286480314, 11.982876967372277)

In [12]:
# prophet model 

def prophet_model(ward_name, train_months):
    dataframe_ward = dataset[dataset['ward_name']==ward_name]
    dataframe = pd.DataFrame()
    dataframe['crimes'] = dataframe_ward.groupby('month').count()['crime_id']
    dataframe['month'] = dataframe_ward.groupby('month')['month'].unique().index
    covid_years = ['2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07']
    for x in covid_years:
        dataframe = dataframe[dataframe['month'] != x]    
    dataframe['month'] = pd.to_datetime(dataframe['month'], format='%Y-%m-%d')
    dataframe = dataframe.rename(columns={'month': 'ds', 'crimes': 'y'})
    # nan_rows = dataframe[dataframe.isnull().any(axis=1)]
    # print(nan_rows)
    # pd.set_option('display.max_rows', None)
    # pd.set_option('display.max_columns', None)
    # print(dataframe)
    
    # Set the uncertainty interval to 95% (the Prophet default is 80%)
    model = Prophet()
    model.fit(dataframe)

    # Create the training dataset, removing the last 'train_months' from the original data
    train = dataframe.drop(dataframe.index[:-train_months])
    # print(train.tail())

    forecast = model.predict(train)
    # print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

    # Calculate MAE between expected and predicted values
    y_true = dataframe['y'][-train_months:].values
    y_pred = forecast['yhat'].values
    # print(y_true, y_pred)

    prophet_r2 = r2_score(y_true, y_pred)
    # print("r^2 Score:", r2)

    prophet_rmse = mean_squared_error(y_true, y_pred, squared=False)
    # print("RMSE:", rmse)

    prophet_mape = mean_absolute_percentage_error(y_true, y_pred)
    # print("mape:", mape)

    return prophet_mape, prophet_rmse, y_pred

# Usage example
ward_name = 'Underhill'
prophet_mape, prophet_rmse, y_pred = prophet_model(ward_name, 6)
prophet_mape, prophet_rmse, y_pred

19:22:47 - cmdstanpy - INFO - Chain [1] start processing
19:22:47 - cmdstanpy - INFO - Chain [1] done processing


(1.1679706639391256,
 2.3592278875179575,
 array([5.56335081, 6.21034876, 4.94361486, 5.76375408, 2.83848408,
        4.76421435]))

In [19]:
# final combined model

def select_model(sarima_mape, sarima_rmse, prophet_mape, prophet_rmse):
    if sarima_rmse < prophet_rmse:
        return "SARIMA"
    else:
        return "Prophet"
    
def combined_model(input_data):
    # Run Prophet model
    prophet_mape, prophet_rmse, y_pred = prophet_model(input_data, 6)
    # print(prophet_mape, prophet_rmse, y_pred)

    # Run SARIMA model
    sarima_mape, sarima_rmse, model_fit= sarima_model(input_data)
    # print(sarima_mape, sarima_rmse, model_fit.forecast()[0])

    # Compare RMSE and select the model
    model_name = select_model(sarima_mape, sarima_rmse, prophet_mape, prophet_rmse)

    if "Prophet" in model_name:
        predictions = y_pred
        mape = prophet_mape
        rmse = prophet_rmse
    elif "SARIMA" in model_name:
        predictions = model_fit.forecast()[0]
        mape = sarima_mape
        rmse = sarima_rmse
    else:
        return 'Error has occured'

    return f"Model: {model_name}\nPredictions: {predictions}\nMAPE: {mape}\nRMSE: {rmse}"

# Usage example

# input_data = input("Please choose a ward: ")
ward_name = 'Underhill'
result = combined_model(ward_name)
print(result)

19:26:17 - cmdstanpy - INFO - Chain [1] start processing
19:26:17 - cmdstanpy - INFO - Chain [1] done processing


Model: SARIMA
Predictions: 2.3142111182860656
MAPE: 0.4203
RMSE: 1.7416932351059529
